In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

/Users/hamsterlord/miniforge3/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
df = pd.read_csv("data/final/final1.csv")
df = df[df['pot_doc'] != '[]']

y = np.array(df['documentid'])
X = np.array(df)

# df["pot_doc"].astype("O")
# df["pot_date"].astype("O")
# df["pot_price"].astype("O")
# df["pot_name"].astype("O")
# df["pot_address"].astype("O")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
x_column = list(df.columns)
x_column.remove('documentid')

X_train_df = pd.DataFrame(data = X_train, columns = df.columns)
X_test_df = pd.DataFrame(data = X_test, columns = df.columns)
y_train_df = pd.DataFrame(data = y_train, columns = ['documentid'])
y_test_df = pd.DataFrame(data = y_test, columns = ['documentid'])

X_train_df= X_train_df.drop(columns="documentid").drop(columns="Unnamed: 0")
X_test_df= X_test_df.drop(columns="documentid").drop(columns="Unnamed: 0")
display(X_test_df)

,pot_doc,pot_name,pot_price,pot_date,pot_address
0,['00d0904732199'],[10],[1],[0],[93]
1,"['00d0267909555', '00d0167381962', '00d0502129...","[12, 12, 12, 12, 12, 10, 2, 2, 12, 12, 12, 2, ...","[4, 2, 4, 3, 4, 2, 3, 2, 3, 2, 3, 3, 4, 4, 4, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[92, 100, 100, 100, 100, 95, 95, 50, 100, 100,..."
2,"['00d0414196980', '00d0614628355', '00d0891802...","[2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2]","[2, 1, 2, 2, 3, 2, 5, 2, 4, 3, 1, 2]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[96, 96, 92, 96, 44, 92, 54, 96, 92, 45, 96, 96]"
3,"['00d0485748096', '00d0542888628', '00d0677299...","[13, 13, 13, 11, 13]","[4, 3, 1, 3, 3]","[0, 0, 1, 0, 0]","[95, 95, 95, 95, 95]"
4,"['00d0624439175', '00d0726523245', '00d0447521...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 13, 2, 2, 13...","[3, 2, 2, 2, 2, 4, 2, 3, 2, 1, 3, 3, 3, 2, 2, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[97, 93, 97, 97, 93, 97, 97, 97, 97, 97, 97, 9..."
...,...,...,...,...,...
94,['00d0124895452'],[10],[0],[1],[93]
95,['00d0339705365'],[14],[1],[1],[100]
96,['00d0388872730'],[14],[5],[0],[92]
97,"['00d0624439175', '00d0726523245', '00d0447521...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 8, 8, 10...","[4, 2, 2, 3, 3, 4, 2, 3, 2, 1, 3, 3, 3, 2, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[97, 93, 97, 97, 93, 97, 97, 97, 97, 97, 97, 9..."


In [23]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*eval(row[1]['pot_price'])[idx])**2 
            + (y*(100-eval(row[1]['pot_date'])[idx])/30)**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/30)**2
            + (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*eval(row[1]['pot_price'])[idx])**2 
            + (y*(100-eval(row[1]['pot_date'])[idx])/30)**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/30)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [24]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    #y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 50)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,1.0,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2023-01-29 04:49:03,419] A new study created in memory with name: no-name-e90c873d-3be9-4c63-955c-4b9ecbc8882c
[I 2023-01-29 04:49:04,252] Trial 0 finished with value: 0.29620253164556964 and parameters: {'x': 2.3948656355567155, 'z': 6.888477396647497}. Best is trial 0 with value: 0.29620253164556964.
[I 2023-01-29 04:49:05,061] Trial 1 finished with value: 0.29620253164556964 and parameters: {'x': 3.880780499451093, 'z': 12.1413774080188}. Best is trial 0 with value: 0.29620253164556964.
[I 2023-01-29 04:49:05,860] Trial 2 finished with value: 0.39746835443037976 and parameters: {'x': 2.1881803376785927, 'z': 40.59552608544703}. Best is trial 0 with value: 0.29620253164556964.
[I 2023-01-29 04:49:06,659] Trial 3 finished with value: 0.29620253164556964 and parameters: {'x': 6.441511861723414, 'z': 18.815634390303888}. Best is trial 0 with value: 0.29620253164556964.
[I 2023-01-29 04:49:07,459] Trial 4 finished with value: 0.4075949367088607 and parameters: {'x': 0.9901155062227984

KeyboardInterrupt: 

In [13]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['z'], 0)
real = list(y_train_df['documentid'])
print(len(res), len(real))

376 376


In [18]:
c = 0
for i in range(len(res)):
    if res[i] != real[i] and real[i] in eval(X_train_df['pot_doc'][i]):
        print(res[i], real[i])
        c += 1
print(c)

00d0726102139 00d0776440475
00d0734493835 00d0286887687
00d0892818805 00d0929882897
00d0617357249 00d0627668053
00d0964499840 00d0288011947
00d0826414442 00d0123031653
00d0551541808 00d0194569644
00d0570374781 00d0453489586
00d0326687304 00d0675769257
00d0472780579 00d0686397126
00d0353042107 00d0801912850
00d0551541808 00d0240488407
00d0531848355 00d0992954979
00d0551541808 00d0572439292
00d0100894168 00d0977582300
00d0743614936 00d0592371977
00d0290184065 00d0803190070
00d0194796076 00d0722335566
00d0375035482 00d0396141876
00d0784103375 00d0980207211
00d0675832982 00d0799986048
00d0626463001 00d0841246769
00d0632354255 00d0439076290
00d0764855917 00d0837787542
00d0733466225 00d0369571763
00d0608126101 00d0571602960
00d0734493835 00d0606540603
00d0726102139 00d0674212079
00d0965540723 00d0330285744
00d0396057665 00d0242626168
00d0340785534 00d0965540723
00d0945445352 00d0726523245
00d0188523936 00d0159517738
00d0206117546 00d0880901672
00d0158328888 00d0639354292
00d0859698635 00d070